In [ ]:
import litellm
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import SequentialAgent, LlmAgent

litellm = LiteLlm(
    model="ollama_chat/gpt-oss:20b",
    api_base="http://localhost:11434",
)

# The first agent generates the initial draft.
generator = LlmAgent(
    name="DraftWriter",
    model=litellm,  # Pass the model to the agent
    description="Generates initial draft content on a given subject.",
    instruction="Write a short, informative paragraph about the user's subject.",
    output_key="draft_text" # The output is saved to this state key.
)

# The second agent critiques the draft from the first agent.
reviewer = LlmAgent(
    name="FactChecker",
    model=litellm,  # Pass the model to the agent
    description="Reviews a given text for factual accuracy and provides a structured critique.",
    instruction="""
    You are a meticulous fact-checker.
    1. Read the text provided in the state key 'draft_text'.
    2. Carefully verify the factual accuracy of all claims.
    3. Your final output must be a dictionary containing two keys:
       - "status": A string, either "ACCURATE" or "INACCURATE".
       - "reasoning": A string providing a clear explanation for your status, citing specific issues if any are found.
    """,
    output_key="review_output" # The structured dictionary is saved here.
)

# The SequentialAgent ensures the generator runs before the reviewer.
review_pipeline = SequentialAgent(
    name="WriteAndReview_Pipeline",
    sub_agents=[generator, reviewer]
)

# Execution Flow:
# 1. generator runs -> saves its paragraph to state['draft_text'].
# 2. reviewer runs -> reads state['draft_text'] and saves its dictionary output to state['review_output'].

In [15]:
from google.adk.runners import InMemoryRunner
from google.genai import types

# Create an InMemoryRunner for the pipeline
runner = InMemoryRunner(
    agent=review_pipeline,
    app_name='reflection_app'
)

# Create a session
session = await runner.session_service.create_session(
    app_name='reflection_app',
    user_id='test_user'
)

# Create the user message with the subject
user_message = types.Content(
    role='user',
    parts=[types.Part.from_text(text="The water in the sea is drinkable by humans with proper treatment.")]
)

# Run the pipeline through the runner
print("Running the reflection pipeline...\n")
async for event in runner.run_async(
    user_id='test_user',
    session_id=session.id,
    new_message=user_message
):
    if event.is_final_response() and event.content and event.content.parts:
        print(f"\n** Final Response from {event.author}:")
        print(event.content.parts[0].text)

App name mismatch detected. The runner is configured with app name "reflection_app", but the root agent was loaded from "/workspace/rnd/agentic-design-patterns/.venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".


Running the reflection pipeline...


** Final Response from DraftWriter:
parts=[Part(
  text='We need to write a short informative paragraph about the subject: "The water in the sea is drinkable by humans with proper treatment." The instruction: "Write a short, informative paragraph about the user\'s subject." So produce paragraph. Keep it concise but informative. Use factual tone. Should mention salinity, treatment methods, purification, desalination, cost, environmental concerns. Ok.',
  thought=True
), Part(
  text='Seawater is not directly potable due to its high salt and mineral content, but it can be made drinkable through various desalination processes. The most common methods are reverse osmosis, thermal distillation, and electrodialysis, which remove dissolved salts, organic matter, and contaminants. Modern desalination plants also integrate advanced filtration, UV disinfection, and mineral re‑addition to improve taste and meet health standards. While these technologies provid